In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import sqlite3
import timeit

# establish sql connection
db_path = "data/input_data.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# PV Comparison

In [4]:
# read in all PV-related data -> interpolation: 1min timesteps / mean: 15min timesteps
resample_method = 'mean'

df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])
df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', conn, parse_dates=['Date'])
df_station40.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43', conn, parse_dates=['Date'])
df_station43.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, parse_dates=['Timestamp'])

if resample_method == 'interpolation':
    df_mb_15 = df_mb_15.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_households.merge(df_mb_15, on='Timestamp', how='inner')
elif resample_method == 'mean':
    df_households = df_households.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_mb_15.merge(df_households, on='Timestamp', how='inner')
else:
    print('WRONG RESAMPLE METHOD!!!')
    raise KeyboardInterrupt

df_results = df_results.merge(df_station40, on='Timestamp', how='inner')
df_results = df_results.merge(df_station43, on='Timestamp', how='inner', suffixes=('_40', '_43'))
df_results['sum_households'] = df_results[['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']].sum(axis=1)

del df_households, df_station40, df_station43, df_mb_15

In [5]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

cols = ['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla', 'sum_households']
for col in cols:
    fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results[col], name=col, opacity=0.7), secondary_y=False)
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['pvpower_instant'] / df_results['pvpower_instant'].max(), name='MeteoBlue',
                         opacity=0.7), secondary_y=True)
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['solar_radiation_40'] / df_results['solar_radiation_40'].max(),
                         name='Wunderground_40', opacity=0.7), secondary_y=True)
fig.add_trace(go.Scatter(x=df_results['Timestamp'], y=df_results['solar_radiation_43'] / df_results['solar_radiation_43'].max(),
                         name='Wunderground_43', opacity=0.7), secondary_y=True)
fig.update_layout(title_text=f'Comparison PV values (resample: {resample_method})', title_x=0.5, template='plotly')
fig.update_yaxes(title_text='Household Batteries', secondary_y=False)
fig.update_yaxes(title_text='MeteoBlue & Wunderground (normalized)', secondary_y=True)
fig.show(renderer='browser')

# PV Correlation

In [ ]:
# read in mb clouds and merge into resampled data

df_mb_clouds = pd.read_sql_query('SELECT * FROM mb_clouds', conn, parse_dates=['Timestamp'])

if resample_method == 'interpolation':
    df_mb_clouds = df_mb_clouds.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
elif resample_method == 'mean':
    df_mb_clouds = df_mb_clouds.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
else:
    print('WRONG RESAMPLE METHOD!!!')
    raise KeyboardInterrupt

df_results = df_results.merge(df_mb_clouds, on='Timestamp', how='inner')

del df_mb_clouds

In [16]:
# correlate every days measured power

corr_target = 'pvpower_instant'
fig = make_subplots(specs=[[{'secondary_y': True}]])

df_corr = pd.DataFrame()
cols = ['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla', 'sum_households']
for col in cols:
    df_corr = pd.concat([df_corr, df_results.groupby(df_results['Timestamp'].dt.date)[col].corr(df_results[corr_target])], axis=1)

df_corr = pd.concat([df_corr, df_results.groupby(df_results['Timestamp'].dt.date)[['lowclouds', 'midclouds', 'highclouds', 'totalcloudcover']].mean()], axis=1)

fig.add_traces([go.Scatter(x=df_corr.index, y=df_corr[col], name=col, opacity=0.7, mode='lines+markers') for col in cols], secondary_ys=[False] * len(cols))
fig.update_yaxes(title_text='Correlation value', range=[0, 1], secondary_y=False)

"""
cols_clouds = ['lowclouds', 'midclouds', 'highclouds', 'totalcloudcover']
fig.add_traces([go.Scatter(x=df_corr.index, y=df_corr[col], name=col, opacity=0.7, mode='lines+markers') for col in cols_clouds],
               secondary_ys=[True] * len(cols_clouds))
fig.update_yaxes(title_text='Cloud cover in %', range=[0, 100], secondary_y=True)

fig.update_layout(title_text='Daily Correlation to MeteoBlue Forecast vs. Cloud Cover (mean)', title_x=0.5,
                  xaxis_title='Date', template='plotly')
"""

fig.update_layout(title_text=f'Daily Correlation to MeteoBlue Forecast (resample: {resample_method})', title_x=0.5,
                  xaxis_title='Date', template='plotly')

fig.show(renderer='browser')

# Comparison parquet

In [ ]:
df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])

df_households.to_parquet('households.parquet')
df_households.to_parquet('households.parquet.gzip', compression='gzip')
df_households.to_csv('households.csv')

del df_households

In [ ]:
setup_sql = 'import sqlite3; import pandas as pd; conn = sqlite3.connect("data/input_data.db")'
setup_pandas = 'import pandas as pd'

t_sql = timeit.Timer("df = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])", setup=setup_sql)
t_parquet = timeit.Timer('df = pd.read_parquet("households.parquet")', setup=setup_pandas)
t_parquet_zip = timeit.Timer('df = pd.read_parquet("households.parquet.gzip")', setup=setup_pandas)
t_csv = timeit.Timer('df = pd.read_csv("households.csv")', setup=setup_pandas)

In [ ]:
print(f'SQL:\n{t_sql.repeat(5, 20)}\n')
print(f'Parquet:\n{t_parquet.repeat(5, 20)}\n')
print(f'Parquet (compressed):\n{t_parquet_zip.repeat(5, 20)}\n')
print(f'CSV:\n{t_csv.repeat(5, 20)}\n')